# What is Selenium?

## 1

What environment do we use?

selenium docker?

See [What is Docker?](What%20is%20Docker.ipynb)



In [1]:
var importer = require('../Core');var fs = require('fs');// add some run commands to the bash scriptvar bashToRun = (code) => code.split('\n').filter(n => n.trim().length > 0).map(l => 'RUN ' + l.trim())    .join('\n').replace(/\\\s*\nRUN\s*/ig, '\\\n ');// create a selenium Dockerfile with a vnc connectionvar seleniumDocker = (outputFile) => {
    const r = importer.interpret([
        'run selenium inside docker',
        // add some extra services
        'linux dev tools',
        'vnc html',
        'vnc docker'
    ]);
    // convert some results to Docker RUN commands instead of bash
    r[1].code = bashToRun(r[1].code);
    r[2].code = bashToRun(r[2].code);
    // save the Dockerfile
    fs.writeFileSync(outputFile, r.map(r => r.code).join('\n'));
    return r;
};module.exports = seleniumDocker;

[Function: seleniumDocker]

How to set up selenium server?

In [2]:
var importer = require('../Core');var path = require('path');var fs = require('fs');var execCmd = importer.import('spawn child process');
var DOWNLOAD_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'Downloads');var PROFILE_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.defaultProfile');var DOCKERFILE = path.resolve(path.join(__dirname, 'Dockerfile'));
var getSeleniumServer = (name = 'act-selenium') => {    try {        fs.mkdirSync(DOWNLOAD_DIR);    } catch (err) {        if (err.code != 'EEXIST') {            throw err;        }    }    try {        fs.mkdirSync(PROFILE_DIR);    } catch (err) {        if (err.code != 'EEXIST') {            throw err;        }    }    try {        fs.unlinkSync(path.join(PROFILE_DIR, 'SingletonLock'));    } catch (err) {        if (err.code != 'ENOENT') {            throw err;        }    }    seleniumDocker(DOCKERFILE);    return execCmd('docker ps -a')
        .then(r => {            if (r[0].indexOf(name) > -1) {                return execCmd('docker stop ' + name)                    .then(r => new Promise(resolve =>                        setTimeout(() => resolve(r), 1000)))                    .then(() => execCmd('docker rm ' + name));            }        })        .then(() => new Promise(resolve =>            setTimeout(() => resolve(), 1000)))        .then(() => {            var build = 'docker build -t ' + name + ' "'                + path.dirname(DOCKERFILE) + '"\n'                + 'docker run --shm-size=3g -d '                + '--name ' + name + ' '                + '-p 8888:8888 '                + '-p 6080:6080 '                + '-p 5900:5900 '                + '-p 4444:4444 '                + '-p 4200:4200 '                + '-p 3000:3000 '                // TODO: add profile dir back in when permissions works on windows                + '-v "' + DOWNLOAD_DIR + '":/data/downloads '                + name + '\n';            return execCmd(build)        })        .then(r => new Promise(resolve => setTimeout(() => resolve(r), 6000)))        .catch(e => console.log(e))};module.exports = getSeleniumServer;getSeleniumServer;

[Function: getSeleniumServer]

In [ ]:
var importer = require('../Core');
var {getSeleniumServer} = importer.import([
    'What is Selenium.ipynb[what is selenium]',
    'set up selenium server'
]);

$$.async();getSeleniumServer()    .then(r => $$.sendResult(r))    .catch(e => $$.sendError(e));

# 2

How to connect to VNC through the web browser?

[Click here to open](http://localhost:6080/vnc.html?password=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true&reconnect=true) in a separate browser


In [ ]:
var urlEncode = (obj) => {    return Object.keys(obj).map(k => encodeURIComponent(k) + '=' + encodeURIComponent(obj[k])).join('&');}var vncIframe = (options = {    password: 'secret',    host: 'localhost',    port: 6080,    autoconnect: true,    resize: 'downscale',    view_only: true,    reconnect: true}) => `<div style="display:block; width:100%; padding-bottom:70%;position:relative;"><iframe id="vnc"style="position:absolute;top:0;right:0;bottom:0;left:0;width:100%;height:100%;border:0;" src="//local:6080/vnc.html?${urlEncode(options)}"></iframe></div>`;module.exports = vncIframe;vncIframe;

In [ ]:
$$.mime({'text/html': vncIframe()});

## 3

How to run a selenium cell on the Docker machine?



In [ ]:
var importer = require('../Core');
var createWebdriverClient = importer.import('webdriver client');

var runSeleniumCell = (search, newWindow = true) => {
    var client = createWebdriverClient();
    var ctx = {client};
    var onlyOneWindow;
    var {
        getCredentials,
        getAllXPath,
        getAllUntil,
        resizeWindow,
        onlyOneWindow,
        getAllSessionUrls,
    } = importer.import([
        'decrypt password',
        'all elements xpath',
        'get all elements until',
        'resize selenium window',
        'only one window',
        'get all session urls'
    ], ctx);
    return (newWindow
            ? client
        .getActiveSessions()
        .then(() => onlyOneWindow())
        .then(() => getAllSessionUrls())
            : client)
        .then(() => {
            return importer.import(search, Object.assign(ctx, {
                browser: client,
                getCredentials,
                getAllXPath,
                getAllUntil,
                resizeWindow
            }))
        })
};
module.exports = runSeleniumCell;


In [ ]:
var importer = require('../Core');
var runSeleniumCell = importer.import('selenium cell');

$$.async()
runSeleniumCell('test docker selenium')
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));


Test docker selenium?

In [ ]:
var testLive = () => {
    return client
        .url('https://purchasesprint.actops.com')
        .click('[href*="/auth"], [routerlink*="/auth"]');
}
module.exports = testLive();




TODO:

Auto fill fields in a live browser using the 

Transfer login state to current browser?

